In [1]:
import sys
import os
from glob import glob
from datasets import Dataset, Features, Value, Sequence
from transformers import AutoTokenizer
import torch

/home/hyle/miniconda3/envs/dl_310/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
sys.path.append(os.path.join(os.getcwd(), ".."))

try:
    from vlsp import read_and_process
    from utils.ner_utils import tokenize_and_align 
except ImportError as e:
    print(e)
    sys.exit(1)

24/11/28 11:37:54 WARN Utils: Your hostname, hyle-nitro resolves to a loopback address: 127.0.1.1; using 192.168.0.216 instead (on interface wlp0s20f3)
24/11/28 11:37:54 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/11/28 11:37:54 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [ ]:
tokenizer = AutoTokenizer.from_pretrained("../tokenizer/trained_tokenizer/tokenizer-50k")


text_files = sorted(glob(os.path.join("../vlsp/data/formatted_data/2016", "**", "*.txt"), recursive=True))
df = read_and_process.read_format_and_return_df(text_files, remove_tags=False)

In [6]:
ds = Dataset.from_spark(df, split="train")
ds = ds.with_format("torch")

label_map = read_and_process.get_label_map()

ds = ds.map(tokenize_and_align, batched=True, num_proc=16, fn_kwargs={"tokenizer": tokenizer, "label_map": label_map})
print(ds)

Map (num_proc=16): 100%|██████████| 19687/19687 [00:05<00:00, 3827.71 examples/s] 

Dataset({
    features: ['words', 'tags', 'input_ids', 'attention_mask', 'token_type_ids', 'labels'],
    num_rows: 19687
})


In [5]:
input_id = ds["input_ids"][0]
attn = ds["attention_mask"][0]
print(type(input_id))
print(attn)

NameError: name 'ds' is not defined

In [ ]:
tag = ds["tags"][0]

print(tag)

['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']
